# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [418]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-04-25 15:16:56--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.17’

iris.csv.17         100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-04-25 15:16:56 (47.5 MB/s) - ‘iris.csv.17’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [419]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [420]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [421]:
data.drop(data[data['variety']=='Virginica'].index,inplace=True)

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [422]:
data['variety']=data['variety'].map({'Setosa':-1,'Versicolor':1})

Tampilkan beberapa data untuk mengecek hasilnya

In [423]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [424]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [425]:
print(data_uji.shape[0])
print(data_latih.shape[0])

20
80


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [426]:
label_latih = data_latih.pop('variety')
label_uji = data_uji.pop('variety')

## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [427]:
def hitung_cost_gradient(W,X,Y,regularization):
  jarak = 1 - (Y* np.dot(X,W))
  dw = np.zeros(len(W))
  if max(0,jarak)==0:
    di=W
  else:
    di = W - (regularization * Y*X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [428]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y = shuffle(data_latih,label_latih,random_state=101)
    for index, x in enumerate(X):
      delta=hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [429]:
W = sgd(data_latih,label_latih)
print(W)

[-0.25772437 -0.6661773   1.14626688  0.53911443]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [430]:
def testing(W,data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi,y_prediksi)
  return prediksi

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [431]:
y_prediksi = testing(W,data_uji)
print(sum(y_prediksi==label_uji))

20


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

In [432]:
data_tugas = pd.read_csv('iris.csv')

In [433]:
data_tugas.drop(data_tugas[data_tugas['variety']=='Setosa'].index, inplace = True)
data_tugas['variety'] = data_tugas['variety'].map({'Virginica':-1, 'Versicolor':1})

In [434]:
data_tugas.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1


In [435]:
data_tugas.variety.unique()

array([ 1, -1])

In [436]:
from sklearn.model_selection import train_test_split
tugas_latih, tugas_uji = train_test_split(data_tugas, test_size =0.2)

In [437]:
print(tugas_uji.shape[0])
print(tugas_latih.shape[0])

20
80


In [438]:
label_tugas_latih = tugas_latih.pop('variety')
label_tugas_uji = tugas_uji.pop('variety')

In [439]:
tugas = sgd(tugas_latih, label_tugas_latih)
tugas_prediksi = testing(tugas, tugas_uji)
print(tugas)
print(sum(tugas_prediksi == label_tugas_uji))

[ 4.81673664  3.41220335 -5.48912668 -7.87504319]
17


Hasilnya tidak lebih baik karena hanya mampu mengenali 17 diantara 20 data uji.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [440]:
tugas_epoch = sgd(tugas_latih, label_tugas_latih, max_epoch = 5000)
tugas_epoch_prediksi = testing(tugas_epoch, tugas_uji)
print(tugas_epoch)
print((sum(tugas_epoch_prediksi == label_tugas_uji)))

[  8.11051637   4.78759065 -10.06146519  -9.27284046]
18


Pada percobaan saya, hasilnya lebih baik (17 dibanding sebelumnya 20) karena ketika max_epoch memiliki nilai 5000, hasilnya data yang terprediksi yaitu 18 dari 20 data uji. Namun, tidak selalu benar apabila memperpanjang jumlah epoch saat melakukan proses training ulang SVM dengan nilai max_epoch sebesar 5000 saat memanggil fungsi SGD akan menghasilkan hasil yang lebih baik. Keputusan harus didasarkan pada analisis jumlah data dan kompleksitas model SVM yang digunakan. Jika keduanya cukup besar, maka penambahan epoch dapat membantu meningkatkan kinerja model. Namun, jika salah satu atau keduanya terbatas, penambahan epoch mungkin tidak menghasilkan hasil yang lebih baik.

Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [441]:
dsl = data_tugas.copy()
dsl.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1


In [442]:
from sklearn.model_selection import train_test_split
dsl_latih, dsl_uji = train_test_split(dsl, test_size = 0.2)
dsl.drop(columns=['petal.length', 'petal.width'], inplace=True)

In [443]:
label_dsl_latih = dsl_latih.pop('variety')
label_dsl_uji = dsl_uji.pop('variety')

In [444]:
wdsl = sgd(dsl_latih, label_dsl_latih)
print(wdsl)

[ 4.5313422   2.51153431 -5.11451396 -6.17545095]


In [445]:
ydsl_prediksi = testing(wdsl, dsl_uji)
print(sum(ydsl_prediksi == label_dsl_uji))

19


Pada percobaan saya, hasilnya lebih baik karena setelah petal.length dan petal.width-nya data sudah didrop maka yang terprediksi 10 dari 20 data uji tetapi lebih tinggi (19 dibanding 17 dan 18). Ini disebabkan berkurangnya fitur yang dapat menyebabkan proses seleksi dan algoritma menjadi kurang tergeneralisasi karena parameter klasifikasi yang lebih sedikit. Namun, dari segi kompleksitas menjadi lebih baik karena proses seleksi fitur yang dipakai hanya dua dari empat, sehingga waktu pengerjaannya lebih cepat